In [31]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory


ModuleNotFoundError: No module named 'langchain_openai'

In [ ]:
import os
import getpass

# Get NVIDIA API key securely (will not display when typing)
nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
os.environ["NVIDIA_API_KEY"] = nvidia_api_key

# Optional: Verify the key is set (will only show length, not the actual key)
print(f"NVIDIA API key is set with length: {len(os.environ['NVIDIA_API_KEY'])}")

In [32]:

# Initialize the LLM
llm = ChatNVIDIA(model="mixtral_8x7b")

# Define specialized tools for each agent
search_tool = DuckDuckGoSearchRun()
tools = [
    Tool(
        name="Web Search",
        func=search_tool.run,
        description="Useful for searching information on the internet"
    )
]



NameError: name 'ChatOpenAI' is not defined

In [15]:
# Define custom prompts for the agents
researcher_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert research agent. Your role is to gather comprehensive, accurate information about given topics.
    Follow these guidelines:
    1. Search for recent and reliable information
    2. Focus on factual data and credible sources
    3. Organize information clearly and logically
    4. Cite sources when possible
    5. Maintain objectivity in your research

    Use the tools provided to conduct your research."""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    ("system", "Here's your workspace for thoughts and actions:"),
    ("assistant", "{agent_scratchpad}")
])

analyst_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert analysis agent. Your role is to analyze and synthesize information, drawing meaningful insights and conclusions.
    Follow these guidelines:
    1. Identify key patterns and trends
    2. Evaluate the significance of findings
    3. Consider implications and potential impacts
    4. Provide balanced perspectives
    5. Make well-reasoned recommendations

    Focus on delivering clear, actionable insights."""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    ("system", "Here's your workspace for thoughts and actions:"),
    ("assistant", "{agent_scratchpad}")
])

In [26]:
# Create memory with the correct key names
researcher_memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

analyst_memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [27]:
# Initialize agents with different roles
researcher = create_openai_functions_agent(
    llm=llm,
    prompt=researcher_prompt,
    tools=tools
)

analyst = create_openai_functions_agent(
    llm=llm,
    prompt=analyst_prompt,
    tools=[]  # Analyst doesn't need search tools
)

In [28]:
# Create executors with memory
researcher_executor = AgentExecutor(
    agent=researcher,
    tools=tools,
    memory=researcher_memory,
    verbose=True
)

analyst_executor = AgentExecutor(
    agent=analyst,
    tools=[],
    memory=analyst_memory,
    verbose=True
)

In [29]:
# Example workflow
def research_and_analyze(query):
    # First, let the researcher gather information
    research_results = researcher_executor.invoke(
        {
            "input": f"Research the following topic: {query}",
            "chat_history": []  # Initialize empty chat history
        }
    )
    
    # Then, let the analyst analyze the findings
    analysis = analyst_executor.invoke(
        {
            "input": f"Analyze these research findings: {research_results['output']}",
            "chat_history": []  # Initialize empty chat history
        }
    )
    
    return {
        "research": research_results['output'],
        "analysis": analysis['output']
    }

In [30]:
results = research_and_analyze("What are the latest developments in quantum computing?")
print("\nResearch Results:")
print(results['research'])
print("\nAnalysis:")
print(results['analysis'])



> Entering new AgentExecutor chain...


Exception: [401] Unauthorized
Authentication failed
Please check or regenerate your API key.